In [1]:
import pandas as pd
import numpy as np
import pickle
import logging
import plotly.express as px
from sklearn.model_selection import train_test_split
from IPython.display import display
from cluster import cluster_movies
import logging
logging_format = '%(asctime)s %(levelname)s: %(message)s'
logging.basicConfig(format=logging_format, datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO)
seed=50000

# Exploración del contenido de la data.

De acuerdo a la documentación disponible de esta base de datos (vía: https://files.grouplens.org/datasets/movielens/ml-20m-README.html) se llega a que:
* *links.cv* no se usará debido a que solo son identificadores de las películas en las páginas.
* *genome-scores.csv* y *genome-tags.csv*. La columna que ofrecería cierta información razonable sería `relavance`, pero, la descripción dice lo siguiente: *"the tag genome was computed using a machine learning algorithm on user-contributed content including tags, ratings, and textual reviews"*. Se entiende que ha utilizado información de toda la data del conjunto, por lo que considerar estos scores como features para el entrenamieto y prueba estaría provocando fuga de información (leakege) de uno a otro conjunto. Entonces, no se usarán estos scores.  

Se cargan las datas de interés y se hacen una serie de depliegues de las primeras filas de cada una para conocer el contenido.

In [3]:
movie = pd.read_csv('movie.csv')
rating = pd.read_csv('rating.csv')
tag = pd.read_csv('tag.csv')
print("BASE DE DATOS: movie")
display(movie.head(3))
display(movie.shape)
print("Presencia de missing:")
display(movie.isna().sum())
print("BASE DE DATOS: rating")
display(rating.head(3))
display(rating.shape)
print("Presencia de missing:")
display(rating.isna().sum())
print("BASE DE DATOS: tag")
display(tag.head(3))
display(tag.shape)
print("Presencia de missing:")
display(tag.isna().sum())

BASE DE DATOS: movie


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


(27278, 3)

Presencia de missing:


movieId    0
title      0
genres     0
dtype: int64

...

BASE DE DATOS: tag


,userId,movieId,tag,timestamp
0,18,4141,Mark Waters,2009-04-24 18:19:40
1,65,208,dark hero,2013-05-10 01:41:18
2,65,353,dark hero,2013-05-10 01:41:19


(465564, 4)

Presencia de missing:


userId        0
movieId       0
tag          16
timestamp     0
dtype: int64

Con base en estadísticios resumen construidos de manera independiente. Se tienen los siguientes insights con respecto a la data.

Para los 138,493 __usuarios__ que aparecen en *rating.csv* se tiene que:
* Todos ellos han revisado como mínimo 20 películas.
* Uno de ellos ha revisado 9,254 películas.  
* Sólo 7,801 han dejado dejado algún tipo de tags en las películas.

Para las 27,278 __películas__ que aparecen en *movies.csv*.
* 19,545 tiene algún tipo de *token*: palabra relevante en los tags obtenidas con ayuda de procesamiento de lenguaje natural.
* Solo aparecen 26,744 en la data *rating.csv*, de las cuales:

    1) 3,972 solo tiene 1 reseña. \
    2) 2,043 solo tiene 2 reseñas. \
    3) 1,355 solo tiene 3 reseñas. \
    ... \
    10) 372 solo tiene 10 reseñas. \
    ... 

La estructura de esta data es natural para Sistemas de Recomendación basados en Filtrado Colaborativo. Además, hay una indexación en el tiempo. Habrá que adaptar la estructura de esta base de datos a una típica de tratar con modelos de aprendizaje supervisado.

Para tal fin, dividiremos los __usuarios__ de una vez en conjuntos de entrenamiento y prueba. Esto hará que la información en el tiempo de cierto usuario en el entrenamiento no aparezca en el test y viceversa.

Con base en lo anterior, __el modelo que se preparará predecirá para nuevos usuarios que comiencen a hacer historial de *rating* dada una lista conocida de películas__.

In [4]:
logging.info('SE DIVIDE EN USUARIOS DE ENTRENAMIENTO Y USUARIOS DE PRUEBA.')
total_user = rating[['userId']].drop_duplicates()
total_user.reset_index(drop=True,
                       inplace=True)
user_train, user_test = train_test_split(total_user,
                                         test_size=0.30,
                                         random_state=seed,
                                         shuffle=True)

2021-11-28 08:38:58 INFO: SE DIVIDE EN USUARIOS DE ENTRENAMIENTO Y USUARIOS DE PRUEBA.


Con el fin de no saturar los recursos computaciones que tenemos a disposición. Tomaremos el 50% de cada uno de esos conjuntos.

In [5]:
logging.info('SE SUBMUESTREAN LOS CONJUNTOS DE USUARIOS DE ENTRENAMIENTO Y DE PRUEBA.')
user_train_1, user_train_2 = train_test_split(user_train,
                                              test_size=0.50,
                                              random_state=seed,
                                              shuffle=True)
user_test_1, user_test_2 = train_test_split(user_test,
                                            test_size=0.50,
                                            random_state=seed,
                                            shuffle=True)
list_user_test_1=list(user_test_1['userId'])
list_user_train_2=list(user_train_2['userId'])
rating_redundant_test_1 = rating[rating['userId'].isin(list_user_test_1)]
rating_redundant_train_2 = rating[rating['userId'].isin(list_user_train_2)]
rating_redundant_test_1.sort_values(by='timestamp',
                                    ascending=False,
                                    inplace=True)
rating_redundant_test_1.reset_index(drop=True,
                                    inplace=True)
rating_redundant_train_2.sort_values(by='timestamp',
                                     ascending=False,
                                     inplace=True)
rating_redundant_train_2.reset_index(drop=True,
                                     inplace=True)

2021-11-28 09:13:48 INFO: SE SUBMUESTREAN LOS CONJUNTOS DE USUARIOS DE ENTRENAMIENTO Y DE PRUEBA.


In [ ]:
rating_redundant_test_1

In [ ]:
rating_redundant_train_2

In [ ]:
len(rating_redundant_test_1['userId'].unique())

In [ ]:
len(rating_redundant_train_2['userId'].unique())

In [ ]:
len(rating_redundant_test_1['movieId'].unique())

In [ ]:
len(rating_redundant_train_2['movieId'].unique())

In [ ]:
# path_test = 'rating_redundant_test_1.sav'
# pickle.dump(rating_redundant_test_1, open(path_test, 'wb'))
# path_train = 'rating_redundant_train_2.sav'
# pickle.dump(rating_redundant_train_2, open(path_train, 'wb'))
# path_test = 'rating_redundant_test.sav'
# rating_redundant_test = pickle.load(open(path_test, 'rb'))
# path_train = 'rating_redundant_train.sav'
# rating_redundant_train = pickle.load(open(path_train, 'rb'))

# Codificación de la variable de interés.

In [ ]:
logging.info('SE CODIFICA VARIABLE OBJETIVO.')
rating_redundant_test_1['high_rating'] = rating_redundant_test_1.apply(lambda row: 1 if row['rating'] >= 4 else 0,
                                                                       axis=1)
rating_redundant_train_2['high_rating'] = rating_redundant_train_2.apply(lambda row: 1 if row['rating'] >= 4 else 0,
                                                                         axis=1)
logging.info('¡LISTO!')      

In [ ]:
logging.info('SE OBTIENE FECHA y AÑO.')
rating_redundant_test_1['time_day'] = rating_redundant_test_1.apply(lambda row: row['timestamp'].split()[0], axis=1)
rating_redundant_test_1['time_day'] = pd.to_datetime(rating_redundant_test_1['time_day'])
rating_redundant_test_1['timestamp'] = pd.to_datetime(rating_redundant_test_1['timestamp'])
rating_redundant_test_1['year'] = pd.DatetimeIndex(rating_redundant_test_1['time_day']).year
rating_redundant_train_2['time_day'] = rating_redundant_train_2.apply(lambda row: row['timestamp'].split()[0], axis=1)
rating_redundant_train_2['time_day'] = pd.to_datetime(rating_redundant_train_2['time_day'])
rating_redundant_train_2['timestamp'] = pd.to_datetime(rating_redundant_train_2['timestamp'])
rating_redundant_train_2['year'] = pd.DatetimeIndex(rating_redundant_train_2['time_day']).year
# rating['time_age'] = rating.apply(lambda row: row['timestamp'].split()[0].replace("-", " ").split()[0], axis=1)
logging.info('¡LISTO!')

In [ ]:
rating_redundant_train_2

In [ ]:
rating_redundant_test_1

Observemos el siguiente ejemplo de usuario que ha caído en el conjunto de entrenmaiento.

In [ ]:
ind_userId=85252
data_ind_user_Id = rating_redundant_train_2[rating_redundant_train_2['userId']==ind_userId]
count_movies_ind_userId = data_ind_user_Id.shape[0]
display(data_ind_user_Id.head(10))
logging.info(f'EN TOTAL EL USUARIO {ind_userId} TIENE {count_movies_ind_userId}.')

Este pasado usuario ha reseñado en total 57 películas (aquí se muestran las primeras 10). Sin embargo, hay películas "muy similares entre sí", por lo que esta manera de usar la información podría dar cierta redundancia que podría incidir en el sobreajuste del modelo.

Siendo así, se agruparán las películas que han quedado en el conjunto de entrenamiento,

Este modo se tratar la data asegura no tener comportamiento de usuario heredados del conjunto de entrenamiento. Tambien se tiene lo siguiente

In [ ]:
movies_in_test = list(rating_redundant_test_1['movieId'].unique())
movies_in_train = list(rating_redundant_train_2['movieId'].unique())
count_not_movies_in_train = len(set(movies_in_test) - set(movies_in_train))
logging.info(f'EN TOTAL HAY {len(movies_in_test)} PELÍCULAS EN EL CONJUNTO DE USUARIOS-TEST.')
logging.info(f'EN TOTAL HAY {len(movies_in_train)} PELÍCULAS EN EL CONJUNTO DE USUARIOS-TRAIN.')
logging.info(f'EN TOTAL HAY {count_not_movies_in_train} PELÍCULAS QUE NO ESTÁN EN EL CONJUNTO USUARIOS-TRAIN.')

En en esta intersección de conjuntos de películas y usuarios que "no verá" el modelo donde debemos interesarnos en el comportamimiento del modelo.

# Clusterización de las películas 

In [ ]:
interest_columns = ['movieId', 'genre_film-noir',
                    'genre_no genres listed', 'genre_drama',
                    'genre_mystery', 'genre_animation',
                    'genre_horror', 'genre_fantasy',
                    'genre_war', 'genre_crime', 'genre_comedy',
                    'genre_western', 'genre_adventure',
                    'genre_documentary', 'genre_imax',
                    'genre_action', 'genre_children',
                    'genre_musical', 'genre_thriller',
                    'genre_romance', 'genre_sci-fi'] + ['cluster']

In [ ]:
movie_with_cluster = cluster_movies(data_movie=movie,
                                    n_clusters=15)

In [ ]:
# data_path='movie_with_cluster.sav'
# pickle.dump(movie_with_cluster, open(data_path, 'wb'))

# Obtención de la base de entrenamiento definitiva

In [ ]:
rating_train_2 = pd.merge(rating_redundant_train_2,
                          movie_with_cluster,
                          how="left",
                          on=["movieId"])
rating_train_2.reset_index(drop=True,
                           inplace=True)

In [ ]:
rating_train_2

In [ ]:
rating_train_whithout_duplicate = rating_train_2.drop_duplicates(subset=['userId', 'high_rating', 'cluster'])

In [ ]:
rating_train_whithout_duplicate

In [ ]:
data_ind = rating_train_whithout_duplicate[rating_train_whithout_duplicate['userId']==87586][['userId', 'timestamp', 'movieId', 'rating', 'high_rating', 'title', 'genres', 'genres_list', 'cluster']]
data_ind

In [ ]:
data_ind['cluster'].value_counts()

In [ ]:
data_ind[data_ind['cluster']==11]

In [ ]:
rating_train_whithout_duplicate

In [ ]:
# data_path='rating_train_whithout_duplicate.sav'
# pickle.dump(rating_train_whithout_duplicate, open(data_path, 'wb'))

# Obtención de la base de prueba definitiva

La base de datos definitva serán los usurios de `rating_redundant_train_2` cuyas películas no estén en la base de entrenamiento.

In [ ]:
movies_not_in_train = list(set(movies_in_test) - set(movies_in_train))

In [ ]:
user_for_movies_not_in_train = list(rating_redundant_test_1[rating_redundant_test_1['movieId'].isin(movies_not_in_train)]['userId'])

In [ ]:
data_test = rating_redundant_test_1[rating_redundant_test_1['userId'].isin(user_for_movies_not_in_train)]

In [ ]:
data_test

In [ ]:
len(data_test[data_test['userId']==79366]['time_day'].unique())

In [ ]:
data_test_with_movies_not_in_train = pd.merge(data_test,
                                              movie_with_cluster,
                                              how="left",
                                              on=["movieId"])
data_test_with_movies_not_in_train.reset_index(drop=True,
                                               inplace=True)

In [ ]:
data_test_with_movies_not_in_train

In [ ]:
rating_redundant_test_1

In [ ]:
# data_path='rating_redundant_test_1.sav'
# pickle.dump(rating_redundant_test_1, open(data_path, 'wb'))

In [ ]:
rating_redundant_test_1

In [ ]:
# data_path='rating_redundant_test_1_movies_not_in_train.sav'
# pickle.dump(data_test, open(data_path, 'wb'))

In [ ]:
data_test

In [ ]:
# data_path='final_data_test.sav'
# d_2 = pickle.load(open(data_path, 'rb'))

In [ ]:
d_2 

In [ ]:
rating_train_whithout_duplicate

In [ ]:
# d = pickle.load(open('final_data_test_complete.sav', 'rb'))

In [ ]:
d 

In [ ]:
d.isna().sum()

In [ ]:
d[d['percencumulativefrecc_genre_film-noir_per_day'].isna()]['userId'].unique()

In [ ]:
rating[rating['userId']==112676]

In [ ]:
d.isna().sum().tail(20)

In [ ]:
d

In [ ]:
d[d['userId']==91577]

In [ ]:
# data_train_no_duplicates = pickle.load(open('rating_train_whithout_duplicate.sav',
#                                             'rb'))

In [ ]:
data_train_no_duplicates